In [598]:
import pandas as pd
import numpy as np
from jupyter_utils import jupyter_setup, load_tracker
jupyter_setup()
import os
from evaluation_and_tracking import IDPerformanceTracker
import torch
from sklearn.metrics import roc_curve, auc, confusion_matrix, precision_recall_curve
from sklearn.linear_model import LinearRegression, LogisticRegression

C:\Michi\acoustic_covid_detection\python


In [545]:
# ID_PERFORMANCE_TRACKING = "test_linearRegression.pickle"
ID_PERFORMANCE_TRACKING = "MILmetadata.pickle"
# ID_PERFORMANCE_TRACKING = "02_MILDDenseNoMetadata.pickle"
# ID_PERFORMANCE_TRACKING = "MILDDenseNoMetadata_2epochs.pickle"

In [546]:
id_performance = IDPerformanceTracker(ID_PERFORMANCE_TRACKING)
id_performance.df = id_performance.load()

In [547]:
len(id_performance)

2888

In [548]:
eval_data = id_performance.df[id_performance.df.set_type == "eval"]
test_data = id_performance.df[id_performance.df.set_type == "test"]

In [549]:
eval_ids = eval_data.ID.unique()
test_ids = test_data.ID.unique()
len(eval_ids), len(test_ids)

(356, 466)

In [550]:
recording_types = eval_data.rec_type.unique()
recording_types

array(['combined_breaths', 'combined_coughs', 'combined_speech',
       'combined_vowels'], dtype=object)

In [551]:
class Participant:
    def __init__(self, identifier, df, allow_n_missing_recordings=2):
        self.id = identifier
        self.cough = self.get_single_prediction("combined_coughs", df)
        self.speech = self.get_single_prediction("combined_speech", df)
        self.breath = self.get_single_prediction("combined_breaths", df)
        self.vowels = self.get_single_prediction("combined_vowels", df)
        # try:
        self.label = df[df.ID == identifier].label.values[0]
        # except IndexError:
            # self.label = df[df.ID == identifier].label.values
        
        no_recording = 0
        if self.cough is None:
            no_recording += 1
            self.cough = 0
        if self.speech is None:
            no_recording += 1
            self.speech = 0
        if self.breath is None:
            no_recording += 1
            self.breath = 0
        if self.vowels is None:
            no_recording += 1
            self.vowels = 0
        if no_recording > allow_n_missing_recordings:
            raise ValueError("there is at least 1 recording not present")
            
            
        
    def get_single_prediction(self, rec_type, df):
        idx = np.logical_and(df.ID == self.id, df.rec_type == rec_type)
        n_entries = len(df[idx])
        if n_entries == 1:
            try:
                prediction = df[idx].prediction.values[0][-1]
            except IndexError:
                prediction = df[idx].prediction.values[0]
                
        elif n_entries == 0:
            # print("error")
            # raise ValueError("No Entry for this")
            # prediction = 0
            prediction = None
        else:
            raise ValueError("there cannot be more than one entry with the same ID and rec type")
        # add sigmoid???
        # print(prediction)
        return prediction
    
    def get_all_predictions(self):
        return np.array([self.cough, self.speech, self.breath, self.vowels])
    # def calculate AUCROC, accuracy, loss for one category and after linear regression?

In [552]:
participant = Participant(eval_ids[0], eval_data)
participant.get_all_predictions()

array([0.10253654, 0.00629406, 0.17058913, 0.16971987])

In [553]:
def get_linregr_matrices(eval_ids, data, allow_n_missing_recordings=2):
    predictions_matrix = np.array([])
    labels = np.array([])
    ids = []
    for i, participant_id in enumerate(eval_ids):
        try:
            participant = Participant(participant_id, data, allow_n_missing_recordings=allow_n_missing_recordings)
            ids.append(participant_id)
        except ValueError:
            # print("error")
            continue
        # print(participant_id)
        if i == 0 or len(predictions_matrix) == 0:
            predictions_matrix = participant.get_all_predictions()
            labels = np.array([participant.label])
        else:
            predictions = participant.get_all_predictions()
            # print(predictions)
            predictions_matrix = np.vstack([predictions_matrix, predictions])
            labels = np.append(labels, participant.label)
        # print(participant_id)
    return predictions_matrix, labels, ids

In [554]:
def get_accuracy(labels, predictions, threshold=0.5, verbose=False):
    labels_bool = labels > threshold
    # predictions = torch.sigmoid(torch.Tensor(predictions))
    predicted_labels = predictions > threshold
    n_correctly_predicted = np.sum(predicted_labels == labels_bool) / len(predictions)
    return np.round(n_correctly_predicted*100, 1)

In [555]:
def get_aucroc(labels, predictions):
    # using mixup, the resulting labels are no longer binary but continous between 0 and 1
    # we round to get any kind of result but for the training data, the auc-roc is not quite meaningful
    # labels = np.round(self.labels)
    # try:
    fpr, tpr, thresh = roc_curve(labels, predictions)
    aucroc = auc(fpr, tpr)
    # except ValueError:
    #     # fpr, tpr, thresh = 0, 0, 0
    #     aucroc = 0.0
    return np.round(aucroc*100, 1)

In [556]:
def get_confusion_matrix_parameters(labels, predictions, threshold=0.5, verbose=False):
    predictions_bool = predictions > threshold
    labels_bool = labels > threshold
    confusion_mat = confusion_matrix(labels_bool, predictions_bool)
    mat = np.flip(confusion_mat)
    mat = np.concatenate([mat, np.expand_dims(mat.sum(axis=1), 1)], axis=1)
    mat = np.concatenate([mat, np.expand_dims(mat.sum(axis=0), 0)], axis=0)
    if verbose:
        print("##########################################################################\n")
        print(pd.DataFrame(mat, columns=["Pred. [+]", "Pred. [-]", "True Total"],
                           index=["True [+]", "True [-]", "Pred. Total"]))

    # returns parameters in the following order: tn, fp, fn, tp
    return confusion_mat.ravel()

def get_rates_from_confusion_matrix(confusion_mat, verbose=False):
    TN, FP, FN, TP = confusion_mat
    total_negatives = TN + FP
    total_positives = FN + TP
    tpr = round(TP / total_positives, 4)  # also known as recall
    tnr = round(TN / total_negatives, 4)
    fnr = round(FN / total_positives, 4)
    fpr = round(FP / total_negatives, 4)
    precision = round(TP / (TP + FP), 4)
    if verbose:
        print(pd.DataFrame(dict(tpr=tpr * 100, fpr=fpr * 100, tnr=tnr * 100, fnr=fnr * 100), index=[0]))
    return tpr, fpr, tnr, fnr, precision

def get_auc_prec_recall(labels, predictions):
    precision, recall, _ = precision_recall_curve(labels, predictions)
    auc_preision_recall = auc(recall, precision)
    return auc_preision_recall

# LINEAR REGRESSION

In [557]:
def sigmoid(A):
    return torch.sigmoid(torch.Tensor(A)).numpy()

In [566]:
def extend_linregr_matrx(A):
    # include further components for the linear regression, like a constant, square of each component, inverse squre, square root,...)
    bias = np.ones((A.shape[0], 1))
    # sign = np.sign(A)
    absolute = np.abs(A)
    squares = np.power(A, 2)
    cubes = np.power(A, 3)
    roots = np.power(absolute, 1/2)
    cuberoots = np.power(absolute, 1/3)
    power_four = np.power(A, 4)
    power_five = np.power(A, 5)
    # return np.concatenate((A, squares, roots), axis=1)    
    return np.concatenate((A, squares, roots, cubes, cuberoots), axis=1)    
    # return np.concatenate((A, bias), axis=1)
    # return A


In [590]:
allow_n_missing_recordings=1
A_val, y_val, _ = get_linregr_matrices(eval_ids, eval_data, allow_n_missing_recordings=allow_n_missing_recordings)
A_test, y_test, _ = get_linregr_matrices(test_ids, test_data, allow_n_missing_recordings=allow_n_missing_recordings)

In [591]:
len(A_val), len(A_test)

(275, 453)

In [592]:
# A_val = sigmoid(A_val)
# A_test = sigmoid(A_test)

In [593]:
A_val = extend_linregr_matrx(A_val)
A_test = extend_linregr_matrx(A_test)

In [594]:
model = LinearRegression().fit(A_val, y_val)

In [595]:
y_val_pred = model.predict(A_val)
get_aucroc(y_val, y_val_pred), get_accuracy(y_val, y_val_pred)

(91.9, 89.1)

In [596]:
y_test_pred = model.predict(A_test)
get_aucroc(y_test, y_test_pred), get_accuracy(y_test, y_test_pred)

(90.7, 91.4)

In [597]:
model.coef_

array([-7.29934299e-03,  7.99226324e-01, -3.19275017e+00, -6.91859464e-01,
       -6.58767784e+00, -1.71163058e+00,  1.21738041e+01,  3.27888265e+00,
        2.60792534e+00,  1.86393506e+00,  2.31532747e+00,  3.08549001e+00,
        1.40546630e+01, -2.84377727e+00, -1.49928849e+01, -3.31409559e+00,
       -1.97886512e+00, -1.09455222e+00, -1.41906828e+00, -2.49340813e+00])

# confusion matrix

In [451]:
mat = get_confusion_matrix_parameters(y_val, y_val_pred, verbose=True)
tpr, _, tnr, _, precision = get_rates_from_confusion_matrix(mat, verbose=True)
recall = tpr
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"\nprecision {round(precision*100, 1)}")
print(f"\nF1: {round(f1_score*100, 1)}")
print(f"AUC pr-recall: {round(get_auc_prec_recall(y_val, y_val_pred)*100, 1)}")

##########################################################################

             Pred. [+]  Pred. [-]  True Total
True [+]            77         33         110
True [-]            15        231         246
Pred. Total         92        264         356
    tpr  fpr   tnr   fnr
0  70.0  6.1  93.9  30.0

precision 83.7

F1: 76.2
AUC pr-recall: 85.4


In [452]:
mat = get_confusion_matrix_parameters(y_test, y_test_pred, verbose=True)
tpr, _, tnr, _, precision = get_rates_from_confusion_matrix(mat, verbose=True)
recall = tpr
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"\nprecision {round(precision*100, 1)}")
print(f"F1: {round(f1_score*100, 1)}")
print(f"AUC pr-recall: {round(get_auc_prec_recall(y_test, y_test_pred)*100, 1)}")

##########################################################################

             Pred. [+]  Pred. [-]  True Total
True [+]            23         37          60
True [-]             4        402         406
Pred. Total         27        439         466
     tpr   fpr    tnr    fnr
0  38.33  0.99  99.01  61.67

precision 85.2
F1: 52.9
AUC pr-recall: 70.1


# Get perofrmance for a single rec type

In [609]:
# AUCROC and accuracy for validation set and test set before linear regression
# keep in mind that IDs that were excluded for various reasons (e.g. audio quality) were set to have a prediction right in the middle (0.5) which decreases the performance, especially accuracy
for rec_type_idx in range(4): 
    pred = torch.sigmoid(torch.Tensor(A_val))[:, rec_type_idx].numpy()
    print("eval set:   ", get_aucroc(y_val, pred), get_accuracy(y_val, pred))
    pred = torch.sigmoid(torch.Tensor(A_test))[:, rec_type_idx].numpy()
    print("test set:   ", get_aucroc(y_test, pred), get_accuracy(y_test, pred))
    print("#################################################")

eval set:    85.3 31.6
test set:    81.3 20.1
#################################################
eval set:    87.9 77.8
test set:    86.6 79.9
#################################################
eval set:    58.3 22.9
test set:    67.8 14.1
#################################################
eval set:    79.9 33.8
test set:    82.6 28.7
#################################################


In [610]:
# rec_type = "combined_coughs"
# rec_type = "combined_speech"
rec_type = "combined_breaths"
# rec_type = "combined_vowels"


In [611]:
id_performance = IDPerformanceTracker(ID_PERFORMANCE_TRACKING)
id_performance.df = id_performance.load()

In [612]:
id_performance.df = id_performance.df[id_performance.df.rec_type == rec_type]
recording_types = id_performance.df.rec_type.unique()
recording_types, len(id_performance.df)

(array(['combined_breaths'], dtype=object), 711)

In [613]:
eval_data = id_performance.df[id_performance.df.set_type == "eval"]
test_data = id_performance.df[id_performance.df.set_type == "test"]
eval_ids = eval_data.ID.unique()
test_ids = test_data.ID.unique()
len(eval_ids), len(test_ids)

(289, 422)

In [614]:
recording_types = test_data.rec_type.unique()
recording_types

array(['combined_breaths'], dtype=object)

In [615]:
A_val, y_val, val_ids_filtered = get_linregr_matrices(eval_ids, eval_data, allow_n_missing_recordings=3)
A_test, y_test, test_ids_filtered = get_linregr_matrices(test_ids, test_data, allow_n_missing_recordings=3)
len(val_ids_filtered), len(test_ids_filtered)

(289, 422)

In [616]:
A_val = extend_linregr_matrx(A_val)
A_test = extend_linregr_matrx(A_test)
model = LinearRegression().fit(A_val, y_val)

In [617]:
y_val_pred = model.predict(A_val)
get_aucroc(y_val, y_val_pred), get_accuracy(y_val, y_val_pred)

(92.3, 85.5)

In [618]:
y_test_pred = model.predict(A_test)
get_aucroc(y_test, y_test_pred), get_accuracy(y_test, y_test_pred)

(86.5, 92.9)

In [619]:
len(y_test_pred), len(test_ids_filtered)

(422, 422)

In [620]:
parts_test = [Participant(participant_id, test_data, allow_n_missing_recordings=3) for participant_id in test_ids_filtered]
parts_eval = [Participant(participant_id, eval_data, allow_n_missing_recordings=3) for participant_id in val_ids_filtered]

In [203]:
new_preds_test = np.array(torch.sigmoid(torch.Tensor(np.array([part.get_single_prediction(rec_type, test_data) for part in parts_test]))))
new_preds_val = np.array(torch.sigmoid(torch.Tensor(np.array([part.get_single_prediction(rec_type, eval_data) for part in parts_eval]))))

# preds = [part.get_single_prediction(rec_type, test_data) for part in parts]
labels_test = np.array([part.label for part in parts_test])
labels_eval = np.array([part.label for part in parts_eval])

In [204]:
for i in np.linspace(-0.3, 0.0, 40):
    print(i)
    print(get_aucroc(labels_eval, new_preds_val+i), " | ", get_accuracy(labels_eval, new_preds_val+i))

-0.3
85.9  |  75.7
-0.2923076923076923
85.9  |  75.7
-0.2846153846153846
85.9  |  75.7
-0.27692307692307694
85.9  |  75.7
-0.2692307692307692
85.9  |  75.7
-0.26153846153846155
85.9  |  75.7
-0.25384615384615383
85.9  |  75.7
-0.24615384615384614
85.9  |  75.7
-0.23846153846153845
85.9  |  75.7
-0.23076923076923078
85.9  |  75.7
-0.22307692307692306
85.9  |  75.7
-0.2153846153846154
85.9  |  75.7
-0.20769230769230768
85.9  |  75.7
-0.2
85.9  |  75.7
-0.1923076923076923
85.9  |  75.7
-0.18461538461538463
85.9  |  75.7
-0.1769230769230769
85.9  |  75.7
-0.16923076923076924
85.9  |  75.7
-0.16153846153846155
85.9  |  76.4
-0.15384615384615385
85.9  |  76.8
-0.14615384615384616
85.9  |  76.8
-0.13846153846153847
85.9  |  76.8
-0.13076923076923078
85.9  |  77.5
-0.12307692307692308
85.9  |  77.9
-0.11538461538461539
85.9  |  78.6
-0.1076923076923077
85.9  |  79.3
-0.1
85.9  |  80.4
-0.09230769230769231
85.9  |  81.4
-0.08461538461538462
85.9  |  82.5
-0.07692307692307693
85.9  |  82.9
-0.06

In [109]:
x = -0.046153846153846156
print(get_aucroc(labels_eval, new_preds_val+x), " | ", get_accuracy(labels_eval, new_preds_val+x))
print(get_aucroc(labels_test, new_preds_test+x), " | ", get_accuracy(labels_test, new_preds_test+x))

84.1  |  84.1
79.8  |  91.0


In [ ]:
for i in np.linspace(-0.3, 0.0, 20):
    # print(x)
    print(get_aucroc(labels_test, new_preds_test+i), " | ", get_accuracy(labels_test, new_preds_test+i))